In [1]:
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
from tensorflow import config as tfconfig
import pandas as pd
import numpy as np
gpus = tfconfig.list_physical_devices('GPU')

if gpus:
    try:
        for gpu in gpus:
            tfconfig.set_logical_device_configuration(gpu, [tfconfig.LogicalDeviceConfiguration(memory_limit=4096)])
    except RuntimeError as e:
        
        print(e)

In [2]:
train_dataset = image_dataset_from_directory('train',
                                             subset='training',
                                             seed=42,
                                             validation_split=0.1,
                                             batch_size=32,
                                             image_size=(128, 128))

validation_dataset = image_dataset_from_directory('train',
                                             subset='validation',
                                             seed=42,
                                             validation_split=0.1,
                                             batch_size=32,
                                             image_size=(128, 128))

Found 24228 files belonging to 2 classes.
Using 21806 files for training.
Found 24228 files belonging to 2 classes.
Using 2422 files for validation.


In [3]:
class_names = train_dataset.class_names
class_names

['0', '1']

In [4]:
model = Sequential()
model.add(Normalization())
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [5]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model_save_path = 'best_model1.h5'
checkpoint_callback = ModelCheckpoint(model_save_path, 
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

history = model.fit(train_dataset, 
                    validation_data=validation_dataset,
                    epochs=15,
                    callbacks=[checkpoint_callback])

Epoch 1/15
682/682 [==============================] - ETA: 0s - loss: 1.1282 - accuracy: 0.8279
Epoch 1: val_accuracy improved from -inf to 0.97853, saving model to best_model1.h5
682/682 [==============================] - 62s 77ms/step - loss: 1.1282 - accuracy: 0.8279 - val_loss: 0.0747 - val_accuracy: 0.9785
Epoch 2/15
681/682 [============================>.] - ETA: 0s - loss: 0.0746 - accuracy: 0.9742
Epoch 2: val_accuracy improved from 0.97853 to 0.99174, saving model to best_model1.h5
682/682 [==============================] - 49s 72ms/step - loss: 0.0746 - accuracy: 0.9742 - val_loss: 0.0304 - val_accuracy: 0.9917
Epoch 3/15
681/682 [============================>.] - ETA: 0s - loss: 0.0559 - accuracy: 0.9834
Epoch 3: val_accuracy did not improve from 0.99174
682/682 [==============================] - 48s 71ms/step - loss: 0.0558 - accuracy: 0.9834 - val_loss: 0.0417 - val_accuracy: 0.9897
Epoch 4/15
681/682 [============================>.] - ETA: 0s - loss: 0.0463 - accuracy: 0.

In [14]:
test_dataset = image_dataset_from_directory('test',
                                             label_mode='int',
                                             batch_size=64,
                                             image_size=(250, 250))

Found 11514 files belonging to 1 classes.


In [44]:
model_save_path = 'best_model1.h5'
model.load_weights(model_save_path)

In [9]:
y_test = model.predict(test_dataset)

180/180 [==============================] - 8s 40ms/step


In [10]:
ans = np.rint(y_test)
ans = ans.astype(int)
ans = pd.DataFrame(ans)[0]

In [40]:
filenames = os.listdir("test/test")

In [ ]:
img = np.array([image.img_to_array(image.load_img("test/test/"+fname, target_size=(128, 128))) for fname in filenames])

In [45]:
ans = model.predict(img)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
ans = np.rint(ans)
ans = ans.astype(int)
ans = pd.DataFrame(ans)[0]

In [ ]:
names = pd.DataFrame(filenames)[0]

In [ ]:
df = pd.concat([names, ans], axis=1)

In [ ]:
df = df.set_axis(['filename','label'], axis=1)

In [ ]:
df.to_csv('ans.csv', sep = ',')